In [1]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np
import datetime

%load_ext autoreload
%autoreload 2


import gspread
import pandas as pd
from fractions import Fraction

# Pastikan permision access di menu "Share" di rubah ke anyone with the link terlebih dahulu, agar bisa mengunakan cara OAuth
# Jika Ingin lebih dibatasi menggunakan cara google service account lebih disarankan

gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '16evYf2mJBkPCh91wGCiY8RXm1fhnomVb1Ub0TmcL5zs'
SHEET_NAME = '44+UK'


In [2]:
# Open the spreadsheet
spreadsheet_source = gc.open_by_key(SHEET_ID)

# Get all sheet names
sheet_names = [sheet.title for sheet in spreadsheet_source.worksheets()]
sheet_names

# get data gsheet

spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()

# print(rows[:5])

# print('==============================')
# df = pd.DataFrame(rows, dtype="string")
zoho_df = pd.DataFrame(rows, dtype="string")
# df['Mobile Number'] = df['Mobile Number'].apply(Fraction)
zoho_df

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

# zoho_df['Departure_Date'] = zoho_df['Departure_Date'].apply(reorder_date_from_yyyy)
count_filter_awal = zoho_df.shape[0]
zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type
0,zcrm_3584690000000404115,2019-01-07 15:08:38,2019-01-06,Harley,Russell,hars_russ7@hotmail.com,447834782590,,1995-01-24,1995,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
1,zcrm_3584690000000404196,2019-01-07 15:08:38,2019-01-07,Tobias,Klitsch,tobias.klitsch@gmail.com,447516721033,,1982-06-01,1982,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
2,zcrm_3584690000000404209,2019-01-07 15:08:38,2019-01-06,Thomas,Wood,wood.thomas99@yahoo.co.uk,447887405046,,1990-09-01,1990,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
3,zcrm_3584690000000427160,2019-01-07 16:58:49,2019-01-04,Alice,Jackson,alice.jacks00n@googlemail.com,447730430044,,2000-08-26,2000,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
4,zcrm_3584690000000427078,2019-01-07 16:58:49,2019-02-01,Gjort,Ghuua,k@hotmail.com,447701296080,,,,...,,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 52140,UK/Europe,New Lead,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26108,zcrm_3584690000554613073,2025-06-02 4:09:41,,Suzanne,Bennett,sbennett@macbethlankester.com,447747025138,,,,...,,,Timeshare Marketing,Other,Web Enquiry,Karma St. Martin's,Karma Resort Destinations - Booking Enquiry,UK/Europe,,Email and Phone
26109,zcrm_3584690000554904003,2025-06-03 23:03:26,,Sian,Gray,littletonsschoolfriendsassociation@hotmail.com,4407897663432,,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26110,zcrm_3584690000555173002,2025-06-05 19:42:14,,Marie,Fulker,mlfgob@gmail.com,4407964979123,,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26111,zcrm_3584690000555180039,2025-06-05 19:52:04,,Jack,Gregory,jackdanielgregory@gmail.com,447795459609,,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone


In [3]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


# ====================== Awal Filter Umum ====================== 

In [4]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:105: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

'Email'

In [5]:
zoho_df.loc[
    (zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] != 'empty'), 
    'Phone'] = zoho_df['Mobile']
zoho_df.loc[zoho_df['Phone'] == zoho_df['Mobile'],'Mobile'] = 'empty'

In [6]:
count_deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

# ====================== Akhir Filter Umum ====================== 

In [7]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type
20171,zcrm_3584690000075706651,2019-07-18 15:53:52,,Julia,Paradine,juliaparadine@gmail.com,447482233032,empty,1971-01-10,1971,...,,Single,Timeshare Marketing,Karma Experience,Edm,General,KE | EDM | w2jan19 | Claim 7 Nights complimentary,UK/Europe,,Email and Phone
20174,zcrm_3584690000075919103,2019-07-18 18:39:24,2019-07-06,Asad Uddin,Mohammed,786asadmohammed@gmail.com,447858889757,empty,1950-12-10,1950,...,,Married,Timeshare Marketing,Karma Experience UK,Facebook Ads,,Week 30 | KHM FB AF,UK/Europe,New Lead,Email and Phone
20251,zcrm_3584690000075919189,2019-07-18 18:39:24,2019-06-29,Laura Nicoleta,Arsine,lau_nic2002@yahoo.ca,447440108768,empty,1975-03-15,1975,...,,Single,Timeshare Marketing,Karma Experience UK,Facebook Ads,,Week 30 | KHM FB AF,UK/Europe,New Lead,Email and Phone
20771,zcrm_3584690000105574529,2019-08-26 18:41:54,,Izy,Soliman,iisoliman@gmail.com,447956462616,empty,,,...,,Married,Timeshare Marketing,Karma Experience,Web Enquiry,General,KE | Web | w2jan18 | Redeem Gifting Room,UK/Europe,,Email and Phone
20819,zcrm_3584690000109198763,2019-08-31 13:54:53,,Carly,Hull,carlylouise04@gmail.com,447538865978,empty,,,...,,Living Together,Timeshare Marketing,Karma Experience,Web Enquiry,General,KE | Web | w2jan18 | Redeem Gifting Room,UK/Europe,,Email and Phone
20826,zcrm_3584690000110246893,2019-09-02 12:28:36,,Kwong,Wong,k7ktw@btinternet.com,447899028323,empty,,,...,,Married,Timeshare Marketing,Karma Experience,Web Enquiry,General,KE | Web | w2jan18 | Redeem Gifting Room,UK/Europe,,Email and Phone
20857,zcrm_3584690000113173636,2019-09-08 14:19:02,,Raja,Hassain Ali,hassan_tcs@hotmail.co.uk,447904486666,empty,,,...,,Living Together,Timeshare Marketing,Karma Experience,Web Enquiry,General,KE | Web | w2jan18 | Redeem Gifting Room,UK/Europe,,Email and Phone
20859,zcrm_3584690000113235768,2019-09-08 17:14:22,,Nuria,Muniesa Berdun,nuriamuniesa@hotmail.com,447532828416,empty,,,...,,Living Together,Timeshare Marketing,Karma Experience,Web Enquiry,General,KE | Web | w2jan18 | Redeem Gifting Room,UK/Europe,,Email and Phone
20882,zcrm_3584690000115053317,2019-09-12 16:01:18,,David,Holdway,joanddaveholdway@gmail.com,447968004543,empty,,,...,,,Timeshare Marketing,Other Bali DB,KE2 (Guest Exit),Sales Deck Jimbaran,KE | Web | Gift Registration v2,UK/Europe,,Email and Phone
20902,zcrm_3584690000116937411,2019-09-15 15:06:38,,Simon,Charlton,scharlton@antrim.ifsgroup.com,447454146920,empty,,,...,,Married,Timeshare Marketing,Karma Experience,Web Enquiry,General,KE | Web | w2jan18 | Redeem Gifting Room,UK/Europe,,Email and Phone


In [8]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type
0,zcrm_3584690000000404115,2019-01-07 15:08:38,2019-01-06,Harley,Russell,hars_russ7@hotmail.com,447834782590,empty,1995-01-24,1995,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
1,zcrm_3584690000000404196,2019-01-07 15:08:38,2019-01-07,Tobias,Klitsch,tobias.klitsch@gmail.com,447516721033,empty,1982-06-01,1982,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
2,zcrm_3584690000000404209,2019-01-07 15:08:38,2019-01-06,Thomas,Wood,wood.thomas99@yahoo.co.uk,447887405046,empty,1990-09-01,1990,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
3,zcrm_3584690000000427160,2019-01-07 16:58:49,2019-01-04,Alice,Jackson,alice.jacks00n@googlemail.com,447730430044,empty,2000-08-26,2000,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
4,zcrm_3584690000000427078,2019-01-07 16:58:49,2019-02-01,Gjort,Ghuua,k@hotmail.com,447701296080,empty,,,...,,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 52140,UK/Europe,New Lead,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26040,zcrm_3584690000554613073,2025-06-02 4:09:41,,Suzanne,Bennett,sbennett@macbethlankester.com,447747025138,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,Karma St. Martin's,Karma Resort Destinations - Booking Enquiry,UK/Europe,,Email and Phone
26041,zcrm_3584690000554904003,2025-06-03 23:03:26,,Sian,Gray,littletonsschoolfriendsassociation@hotmail.com,4407897663432,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26042,zcrm_3584690000555173002,2025-06-05 19:42:14,,Marie,Fulker,mlfgob@gmail.com,4407964979123,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26043,zcrm_3584690000555180039,2025-06-05 19:52:04,,Jack,Gregory,jackdanielgregory@gmail.com,447795459609,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone


In [9]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type


In [10]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [11]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [12]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [13]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type


In [14]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type
0,zcrm_3584690000000404115,2019-01-07 15:08:38,2019-01-06,Harley,Russell,hars_russ7@hotmail.com,447834782590,empty,1995-01-24,1995,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
1,zcrm_3584690000000404196,2019-01-07 15:08:38,2019-01-07,Tobias,Klitsch,tobias.klitsch@gmail.com,447516721033,empty,1982-06-01,1982,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
2,zcrm_3584690000000404209,2019-01-07 15:08:38,2019-01-06,Thomas,Wood,wood.thomas99@yahoo.co.uk,447887405046,empty,1990-09-01,1990,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
3,zcrm_3584690000000427160,2019-01-07 16:58:49,2019-01-04,Alice,Jackson,alice.jacks00n@googlemail.com,447730430044,empty,2000-08-26,2000,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
4,zcrm_3584690000000427078,2019-01-07 16:58:49,2019-02-01,Gjort,Ghuua,k@hotmail.com,447701296080,empty,,,...,,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 52140,UK/Europe,New Lead,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26040,zcrm_3584690000554613073,2025-06-02 4:09:41,,Suzanne,Bennett,sbennett@macbethlankester.com,447747025138,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,Karma St. Martin's,Karma Resort Destinations - Booking Enquiry,UK/Europe,,Email and Phone
26041,zcrm_3584690000554904003,2025-06-03 23:03:26,,Sian,Gray,littletonsschoolfriendsassociation@hotmail.com,4407897663432,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26042,zcrm_3584690000555173002,2025-06-05 19:42:14,,Marie,Fulker,mlfgob@gmail.com,4407964979123,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26043,zcrm_3584690000555180039,2025-06-05 19:52:04,,Jack,Gregory,jackdanielgregory@gmail.com,447795459609,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone


In [15]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type


In [16]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type
0,zcrm_3584690000000404115,2019-01-07 15:08:38,2019-01-06,Harley,Russell,hars_russ7@hotmail.com,447834782590,empty,1995-01-24,1995,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
1,zcrm_3584690000000404196,2019-01-07 15:08:38,2019-01-07,Tobias,Klitsch,tobias.klitsch@gmail.com,447516721033,empty,1982-06-01,1982,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
2,zcrm_3584690000000404209,2019-01-07 15:08:38,2019-01-06,Thomas,Wood,wood.thomas99@yahoo.co.uk,447887405046,empty,1990-09-01,1990,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
3,zcrm_3584690000000427160,2019-01-07 16:58:49,2019-01-04,Alice,Jackson,alice.jacks00n@googlemail.com,447730430044,empty,2000-08-26,2000,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
4,zcrm_3584690000000427078,2019-01-07 16:58:49,2019-02-01,Gjort,Ghuua,k@hotmail.com,447701296080,empty,,,...,,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 52140,UK/Europe,New Lead,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26040,zcrm_3584690000554613073,2025-06-02 4:09:41,,Suzanne,Bennett,sbennett@macbethlankester.com,447747025138,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,Karma St. Martin's,Karma Resort Destinations - Booking Enquiry,UK/Europe,,Email and Phone
26041,zcrm_3584690000554904003,2025-06-03 23:03:26,,Sian,Gray,littletonsschoolfriendsassociation@hotmail.com,4407897663432,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26042,zcrm_3584690000555173002,2025-06-05 19:42:14,,Marie,Fulker,mlfgob@gmail.com,4407964979123,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26043,zcrm_3584690000555180039,2025-06-05 19:52:04,,Jack,Gregory,jackdanielgregory@gmail.com,447795459609,empty,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone


In [17]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [18]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [19]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Phone'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Phone'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Phone'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [20]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

68

In [21]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 26113
Jumlah Format Email Salah : 13
Jumlah Setelah Clean Format Email Salah : 26100
Jumlah Duplicate (Phone, Mobile dan Email) : 55
Jumlah Setelah Clean Duplicate : 26045
Jumlah Lead Test : 0
Jumlah Setelah Clean Lead Test : 26045
Jumlah Email OTA : 0
Jumlah Setelah Clean Email OTA : 26045
Jumlah Email Karma : 0
Jumlah Setelah Clean Email Karma : 26045
Jumlah Wash Member : 0
Jumlah Wash Booking : 1
Jumlah Setelah Wash Member : 26044
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 0
Jumlah Setelah Wash Phone Email Blank : 26044
Contact Type
Jumlah Email Only : 0
Jumlah Phone Only : 0
Jumlah Email dan Phone : 26044


In [22]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 26113
Jumlah Wash Dedup : 68
Jumlah Wash Member : 0
Jumlah Wash Booking : 1
Jumlah Data Akhir : 26044


In [23]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['First Name'] = zoho_df['First Name'].str.title()
zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

# zoho_df.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\KLOM.csv', index=False)
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\newsletter-clean.xlsx', index=False)

In [24]:
zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type
0,zcrm_3584690000000404115,2019-01-07 15:08:38,2019-01-06,Harley,Russell,hars_russ7@hotmail.com,447834782590,,1995-01-24,1995,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
1,zcrm_3584690000000404196,2019-01-07 15:08:38,2019-01-07,Tobias,Klitsch,tobias.klitsch@gmail.com,447516721033,,1982-06-01,1982,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
2,zcrm_3584690000000404209,2019-01-07 15:08:38,2019-01-06,Thomas,Wood,wood.thomas99@yahoo.co.uk,447887405046,,1990-09-01,1990,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
3,zcrm_3584690000000427160,2019-01-07 16:58:49,2019-01-04,Alice,Jackson,alice.jacks00n@googlemail.com,447730430044,,2000-08-26,2000,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
4,zcrm_3584690000000427078,2019-01-07 16:58:49,2019-02-01,Gjort,Ghuua,k@hotmail.com,447701296080,,,,...,,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 52140,UK/Europe,New Lead,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26039,zcrm_3584690000554613073,2025-06-02 4:09:41,,Suzanne,Bennett,sbennett@macbethlankester.com,447747025138,,,,...,,,Timeshare Marketing,Other,Web Enquiry,Karma St. Martin's,Karma Resort Destinations - Booking Enquiry,UK/Europe,,Email and Phone
26040,zcrm_3584690000554904003,2025-06-03 23:03:26,,Sian,Gray,littletonsschoolfriendsassociation@hotmail.com,4407897663432,,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26041,zcrm_3584690000555173002,2025-06-05 19:42:14,,Marie,Fulker,mlfgob@gmail.com,4407964979123,,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26042,zcrm_3584690000555180039,2025-06-05 19:52:04,,Jack,Gregory,jackdanielgregory@gmail.com,447795459609,,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone


In [26]:
display(zoho_df)
# Update keseluruhan data, delete data yang lama terlebih dahulu
# clear all row before update
worksheet.clear()
worksheet.update([zoho_df.columns.values.tolist()] + zoho_df.values.tolist())

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Card Type,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Contact Type
0,zcrm_3584690000000404115,2019-01-07 15:08:38,2019-01-06,Harley,Russell,hars_russ7@hotmail.com,447834782590,,1995-01-24,1995,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
1,zcrm_3584690000000404196,2019-01-07 15:08:38,2019-01-07,Tobias,Klitsch,tobias.klitsch@gmail.com,447516721033,,1982-06-01,1982,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
2,zcrm_3584690000000404209,2019-01-07 15:08:38,2019-01-06,Thomas,Wood,wood.thomas99@yahoo.co.uk,447887405046,,1990-09-01,1990,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
3,zcrm_3584690000000427160,2019-01-07 16:58:49,2019-01-04,Alice,Jackson,alice.jacks00n@googlemail.com,447730430044,,2000-08-26,2000,...,,,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,,UK/Europe,New Lead,Email and Phone
4,zcrm_3584690000000427078,2019-01-07 16:58:49,2019-02-01,Gjort,Ghuua,k@hotmail.com,447701296080,,,,...,,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 52140,UK/Europe,New Lead,Email and Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26039,zcrm_3584690000554613073,2025-06-02 4:09:41,,Suzanne,Bennett,sbennett@macbethlankester.com,447747025138,,,,...,,,Timeshare Marketing,Other,Web Enquiry,Karma St. Martin's,Karma Resort Destinations - Booking Enquiry,UK/Europe,,Email and Phone
26040,zcrm_3584690000554904003,2025-06-03 23:03:26,,Sian,Gray,littletonsschoolfriendsassociation@hotmail.com,4407897663432,,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26041,zcrm_3584690000555173002,2025-06-05 19:42:14,,Marie,Fulker,mlfgob@gmail.com,4407964979123,,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone
26042,zcrm_3584690000555180039,2025-06-05 19:52:04,,Jack,Gregory,jackdanielgregory@gmail.com,447795459609,,,,...,,,Timeshare Marketing,Other,Web Enquiry,,Form Contact Us,UK/Europe,,Email and Phone


{'spreadsheetId': '16evYf2mJBkPCh91wGCiY8RXm1fhnomVb1Ub0TmcL5zs',
 'updatedRange': "'44+UK'!A1:AB26045",
 'updatedRows': 26045,
 'updatedColumns': 28,
 'updatedCells': 729260}